<a href="https://colab.research.google.com/github/FlyFish0203/flyfish/blob/main/HW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **題目**

尋找高雄前十名的好吃店家

# **摘要**

透過網路爬蟲，尋找PTT中所2025年推薦最多的高雄前十名的店家

# **引言**

因為很好吃，所以需要找出來!!!

來還有的是因為KKDAY嘗試了好幾次，複製了好多的網址，還是沒有辦法跑出來，所以改成靜態的網頁，來練習。

# **方法**

**目標網站描述**

- 目標網站：KKday 高雄美食推薦頁面

- 網址：https://www.kkday.com/zh-tw/category/tw-kaohsiung/restaurants/list?srsltid=AfmBOoq9oQMchGPfloK4Tg6IM50KDO9FW2B7WfH62yUXX70D46iWjB-i

- 頁面結構：包含體驗名稱、圖片、價格、評價等資訊

**工具**

- 使用 Python 的 `requests` 庫發送 HTTP 請求

- 使用 `BeautifulSoup` 解析 HTML 結構

- 使用 Google Chrome 開發者工具觀察資料載入機制（若為 AJAX，則觀察 API 呼叫）

# **嘗試使用kkday來操作**

In [21]:
# 安裝 Selenium
!pip install -q selenium
!apt-get update -qq
!apt-get install -qq chromium-chromedriver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
typeguard 4.4.3 requires typing_extensions>=4.14.0, but you have typing-extensions 4.13.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_mach

In [28]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# ✅ 設定 headless Chrome（無畫面模式）
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ✅ 建立驅動器
driver = webdriver.Chrome(options=options)

# ✅ 開啟 KKday 高雄美食主題頁面
url = "https://www.kkday.com/zh-tw/dcs_api/ajax_get_dynamic_config_variant"
driver.get(url)
time.sleep(6)  # ⏳ 等待頁面與 JS 載入

# ✅ 滾動頁面觸發載入（多次觸發保險）
for _ in range(3):
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(2)

# ✅ 抓取卡片列表（動態 class 名稱為 .product-card-item）
cards = driver.find_elements(By.CSS_SELECTOR, ".product-card-item")[:10]

results = []
for card in cards:
    try:
        name = card.find_element(By.CSS_SELECTOR, ".product-card-item-title").text
        price = card.find_element(By.CSS_SELECTOR, ".currency-price").text
        link = card.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        try:
            rating = card.find_element(By.CSS_SELECTOR, ".rating-score").text
        except:
            rating = "無評價"
        results.append({
            "名稱": name,
            "價格": price,
            "評價": rating,
            "連結": link
        })
    except Exception as e:
        print("⚠️ 卡片擷取錯誤：", e)

driver.quit()

# ✅ 建立 DataFrame 顯示結果
df = pd.DataFrame(results)
if df.empty:
    print("⚠️ 沒抓到任何資料，請檢查網頁結構或網路延遲。")
else:
    df.to_csv("kaohsiung_kkday_top10.csv", index=False)
    print("✅ 已成功擷取 KKday 高雄美食前10名")
    display(df)



⚠️ 沒抓到任何資料，請檢查網頁結構或網路延遲。


#**遇到問題**
- API試了好幾組還是抓不到
- 網址一直說錯誤，還跑出403的提示
- 後來被kkday鎖住了


# **PTT操作的程式碼**

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

# ✅ 建立 PTT session 並通過 over18 驗證
session = requests.Session()
session.post("https://www.ptt.cc/ask/over18", data={"yes": "yes"})

base_url = "https://www.ptt.cc"
start_url = "https://www.ptt.cc/bbs/Food/index.html"

articles = []

page_count = 5  # 你可以爬多一點頁數
for _ in range(page_count):
    res = session.get(start_url)
    soup = BeautifulSoup(res.text, "html.parser")

    for item in soup.select(".r-ent"):
        try:
            title_tag = item.select_one(".title a")
            if not title_tag:
                continue

            title = title_tag.text.strip()
            link = base_url + title_tag["href"]

            if "高雄" not in title:
                continue  # 只抓包含「高雄」的文章

            # 進入內文抓時間
            article_res = session.get(link)
            article_soup = BeautifulSoup(article_res.text, "html.parser")
            meta = article_soup.select(".article-meta-value")
            if len(meta) >= 4:
                date_str = meta[3].text.strip()
                post_time = datetime.strptime(date_str, "%a %b %d %H:%M:%S %Y")
                if post_time.year < 2025:
                    continue  # 過濾 2025 年以前文章

                articles.append({
                    "標題": title,
                    "連結": link,
                    "時間": post_time.strftime("%Y-%m-%d %H:%M")
                })

                print(f"✅ 收錄：{title}")
            time.sleep(0.5)

        except Exception as e:
            print("⚠️ 擷取失敗：", e)
            continue

    # 找上一頁
    prev = soup.select_one("a.btn.wide:contains('上頁')")
    if prev:
        start_url = base_url + prev["href"]
    else:
        break

# 輸出結果
df = pd.DataFrame(articles)
df.to_csv("ptt_kaohsiung_food_2025.csv", index=False)
print("\n📦 共擷取文章數：", len(df))
df


✅ 收錄：[食記] 高雄 三民市場 三水關東煮


/usr/local/lib/python3.11/dist-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


✅ 收錄：[食記] 高雄 A-Grill Steak House 乾式熟成牛排

📦 共擷取文章數： 2


,標題,連結,時間
0,[食記] 高雄 三民市場 三水關東煮,https://www.ptt.cc/bbs/Food/M.1750083062.A.214...,2025-06-16 22:10
1,[食記] 高雄 A-Grill Steak House 乾式熟成牛排,https://www.ptt.cc/bbs/Food/M.1749474770.A.6C1...,2025-06-09 21:12


# **心得**
又更了解到靜態HTML跟動態載入的差別，並且在操作下來，雖然沒有成功做出kkday的東西，但把找資料的地方記起來，可能之後再做多一點修改會更好，但在曾對爬蟲HTML網站的方式，有深刻的記起來。